## <font color='darkblue'>Introduction</font>
([article source](https://towardsdatascience.com/automate-stacking-in-python-fc3e7834772e)) **Utilizing stacking** (<font color='brown'>stacked generalizations</font>) **is a very hot topic when it comes to pushing your machine learning algorithm to new heights**. For instance, most if not all winning Kaggle submissions nowadays make use of some form of stacking or a variation of it. First introduced in [the 1992 paper Stacked Generalization by David Wolpert](https://www.researchgate.net/publication/222467943_Stacked_Generalization), **their main purpose is to reduce the [generalization error](https://en.wikipedia.org/wiki/Generalization_error).** According to Wolpert, they can be understood “as a more sophisticated version of cross-validation”. While Wolpert himself noted at the time that large parts of stacked generalizations are “black art”, it seems that **building larger and larger stacked generalizations win over smaller stacked generalizations. However, as these models keep increasing in size, they also increase in complexity**. Automating the process of building different architectures would significantly simplify this process. The remainder of this article will deal with the [**package vecstack**](https://github.com/vecxoz/vecstack) I recently came across that is attempting just this.

## <font color='darkblue'>What Does a Stacked Generalization Look like?</font>
**The main idea behind the structure of a stacked generalization is to use one or more first level models, make predictions using these models and then use these predictions as features to fit one or more second level models on top.** To avoid overfitting, cross-validation is usually used to predict the OOF (out-of-fold) part of the training set. There are two different variants available in this package but I’m going to describe ‘Variant A’ in this paragraph. To get the final predictions in this variant, we take the mean or mode of all of our predictions. The whole process can be visualized using this GIF from vecstacks’ documentation:

![image](Images/1.gif)
<br/>

## <font color='darkblue'>Use Case: Building a Stacked Generalization for Classification</font>
After having taken a look at the documentation, it was time to try using the package myself and see how it works. To do so, I decided to use the [**wine data set available on the UCI Machine Learning Repository**](https://archive.ics.uci.edu/ml/datasets/wine). The problem statement for this data set is to use the 13 features, which all represent different aspects of the wine, to predict from which of three cultivars in Italy the wine was derived.

To get started, let’s first import the packages we are going to need for our project:

In [4]:
# !pip install vecstack

In [3]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from vecstack import stacking

Now we are ready to import our data and take a look at it to gain a better understanding of what it looks like:

In [5]:
link = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data'
names = ['Class', 'Alcohol', 'Malic acid', 'Ash',
         'Alcalinity of ash' ,'Magnesium', 'Total phenols',
         'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins', 
         'Color intensity', 'Hue', 'OD280/OD315 of diluted wines',
         'Proline']
df = pd.read_csv(link, header=None, names=names)
df.sample(5)

,Class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
41,1,13.41,3.84,2.12,18.8,90,2.45,2.68,0.27,1.48,4.28,0.91,3.00,1035
53,1,13.77,1.90,2.68,17.1,115,3.00,2.79,0.39,1.68,6.30,1.13,2.93,1375
65,2,12.37,1.21,2.56,18.1,98,2.42,2.65,0.37,2.08,4.60,1.19,2.30,678
10,1,14.10,2.16,2.30,18.0,105,2.95,3.32,0.22,2.38,5.75,1.25,3.17,1510
26,1,13.39,1.77,2.62,16.1,93,2.85,2.94,0.34,1.45,4.80,0.92,3.22,1195


Note that I used [.sample()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html) instead if <font color='blue'>.head()</font> to avoid being potentially misled by assuming the entire data set has the structure of the first five rows. Luckily, this data set does not have any missing values, so we can easily use it to test our package right away without any of the usually required data cleaning and preparation.

Following this, we are going to **separate the response from the input variables and perform an 80:20 train-test-split following the example on vecstacks’ documentation.**

In [6]:
y = df[['Class']]
X = df.iloc[:,1:]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

We are getting closer to the interesting part. Remember the GIF from earlier? It is time now to **define a few first level models for our stacked generalization**. This step definitely deserves its own article but for purposes of simplicity, we are going to use three models: A [**KNN-Classifier**](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html), a [**Random Forest Classifier**](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) and an [**XGBoost Classifier**](https://xgboost.readthedocs.io/en/latest/python/python_api.html).

In [7]:
models = [
    KNeighborsClassifier(n_neighbors=5,
                        n_jobs=-1),
        
    RandomForestClassifier(random_state=0, n_jobs=-1, 
                           n_estimators=100, max_depth=3),
        
    XGBClassifier(random_state=0, n_jobs=-1, learning_rate=0.1, 
                  n_estimators=100, max_depth=3)
]

**These parameters were not tuned prior to setting them as the purpose of this article is testing the package**. If you were to optimize performance, you should not just copy and paste these.

Taking the next part of code from the documentation, we are essentially **performing the GIF’s first part using first level models to make predictions:**

In [8]:
S_train, S_test = stacking(
    models,                   
    X_train, y_train, X_test,   
    regression=False,
    mode='oof_pred_bag', 
    needs_proba=False,
    save_dir=None, 
    metric=accuracy_score, 
    n_folds=4, 
    stratified=True,
    shuffle=True,  
    random_state=0,    
    verbose=2
)

c:\users\johnlee\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


task:         [classification]
n_classes:    [3]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [KNeighborsClassifier]
    fold  0:  [0.69444444]
    fold  1:  [0.63888889]
    fold  2:  [0.62857143]
    fold  3:  [0.65714286]
    ----
    MEAN:     [0.65476190] + [0.02509117]
    FULL:     [0.65492958]

model  1:     [RandomForestClassifier]
    fold  0:  [0.97222222]
    fold  1:  [0.97222222]
    fold  2:  [0.97142857]
    fold  3:  [1.00000000]
    ----
    MEAN:     [0.97896825] + [0.01214701]
    FULL:     [0.97887324]

model  2:     [XGBClassifier]
    fold  0:  [0.94444444]
    fold  1:  [0.94444444]
    fold  2:  [0.97142857]
    fold  3:  [1.00000000]
    ----
    MEAN:     [0.96507937] + [0.02297479]
    FULL:     [0.96478873]



The [**stacking function**](https://github.com/vecxoz/vecstack/blob/master/vecstack/core.py#L133) takes several inputs:
* **models:** the first level models we defined earlier
* **X_train, y_train, X_test:** our data
* **regression:** Boolean indicating whether we want to use the function for regression. In our case set to False since this is a classification
* **mode:** using the earlier describe out-of-fold during cross-validation
* **needs_proba:** Boolean indicating whether you need the probabilities of class labels
* **save_dir:** save the result to directory Boolean
* **metric:** what evaluation metric to use (<font color='brown'>we imported the `accuracy_score` in the beginning</font>)
* **n_folds**: how many folds to use for cross-validation
* **stratified**: whether to use stratified cross-validation
* **shuffle**: whether to shuffle the data
* **random_state**: setting a random state for reproducibility
* **verbose**: 2 here refers to printing all info

Again, referring to the GIF, **all that’s left to do now is fit the second level model(s) of our choice on our predictions to make our final predictions.** In our case, we are going to use an [**XGBoost Classifier**](https://xgboost.readthedocs.io/en/latest/python/python_api.html). This step is not significantly different from a regular fit-and-predict in sklearn except for the fact that instead of using `X_train` to train our model, we are using our predictions `S_train`.

In [9]:
model = XGBClassifier(random_state=0, n_jobs=-1, learning_rate=0.1, n_estimators=100, max_depth=3)
    
model = model.fit(S_train, y_train)
y_pred = model.predict(S_test)
print('Final prediction score: [%.8f]' % accuracy_score(y_test, y_pred))

Final prediction score: [0.94444444]


c:\users\johnlee\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


## <font color='darkblue'>Conclusion</font>
Using [**vecstacks’ stacking automation**](https://github.com/vecxoz/vecstack), we’ve managed to predict the correct wine cultivar with an accuracy of approximately 94.4%! As you can see, the API does not collide with the sklearn API and could, therefore, provide **a helpful tool when trying to speed up your stacking workflow**.

As always, if you have any feedback or found mistakes, please don’t hesitate to reach out to me.